In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import re
import plotly.express as px
import requests
from bs4 import BeautifulSoup as BS
from io import StringIO
%matplotlib inline

In [19]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

In [38]:
payload = {'format' :'csv',
          'starttime':'1900-01-01',
          'minlatitude':'34.9884',
          'maxlatitude':'36.6871',
          'minlongitude':'-90.3131',
          'maxlongitude':'-81.6518',
          'limit': [1,20000]}

In [39]:
r = requests.get(url=url, params=payload)

In [40]:
r.text

'time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource\n2013-12-31T14:57:13.490Z,36.2573333,-89.4436667,0.39,2.1,md,17,54,0.03297,0.11,nm,nm610146,2016-04-07T19:05:19.610Z,"4km E of Ridgely, Tennessee",earthquake,0.5,0.2,0.28,7,reviewed,nm,nm\n2013-12-29T14:20:17.510Z,36.536,-89.715,1.4,1.7,md,17,65,0.03753,0.07,nm,nm610143,2016-04-07T19:05:00.700Z,"10km SW of Lilbourn, Missouri",earthquake,0.4,4.3,0.131,4,reviewed,nm,nm\n2013-12-29T00:04:05.610Z,36.311,-89.5293333,8.93,0.9,md,10,116,0.03274,0.02,nm,nm610142,2016-04-07T19:04:54.380Z,"6km NW of Ridgely, Tennessee",earthquake,0.9,2.2,0.613,4,reviewed,nm,nm\n2013-12-28T14:37:16.170Z,36.3131667,-89.5288333,8.99,1.3,md,19,64,0.03226,0.04,nm,nm610141,2016-04-07T19:04:47.810Z,"6km NNW of Ridgely, Tennessee",earthquake,0.4,0.8,0.622,10,reviewed,nm,nm\n2013-12-28T14:36:58.510Z,36.3096667,-89.5323333,8.55,0.6,md,12,87,0.0312,0.03,nm,nm610140

In [41]:
usgs_data = pd.read_csv(StringIO(r.text))

In [42]:
usgs_data

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2013-12-31T14:57:13.490Z,36.257333,-89.443667,0.39,2.1,md,17.0,54.0,0.03297,0.11,...,2016-04-07T19:05:19.610Z,"4km E of Ridgely, Tennessee",earthquake,0.5,0.2,0.280,7.0,reviewed,nm,nm
1,2013-12-29T14:20:17.510Z,36.536000,-89.715000,1.40,1.7,md,17.0,65.0,0.03753,0.07,...,2016-04-07T19:05:00.700Z,"10km SW of Lilbourn, Missouri",earthquake,0.4,4.3,0.131,4.0,reviewed,nm,nm
2,2013-12-29T00:04:05.610Z,36.311000,-89.529333,8.93,0.9,md,10.0,116.0,0.03274,0.02,...,2016-04-07T19:04:54.380Z,"6km NW of Ridgely, Tennessee",earthquake,0.9,2.2,0.613,4.0,reviewed,nm,nm
3,2013-12-28T14:37:16.170Z,36.313167,-89.528833,8.99,1.3,md,19.0,64.0,0.03226,0.04,...,2016-04-07T19:04:47.810Z,"6km NNW of Ridgely, Tennessee",earthquake,0.4,0.8,0.622,10.0,reviewed,nm,nm
4,2013-12-28T14:36:58.510Z,36.309667,-89.532333,8.55,0.6,md,12.0,87.0,0.03120,0.03,...,2016-04-07T19:04:41.380Z,"6km NW of Ridgely, Tennessee",earthquake,0.8,1.4,0.628,8.0,reviewed,nm,nm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6393,1916-02-21T23:39:00.000Z,35.500000,-82.500000,NaN,5.2,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,North Carolina,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc
6394,1915-12-07T18:40:00.000Z,36.000000,-90.000000,NaN,4.5,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,Arkansas,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,bar
6395,1913-03-28T21:50:00.000Z,36.200000,-83.700000,NaN,4.1,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,eastern Tennessee,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc
6396,1903-11-04T19:14:00.000Z,36.500000,-89.800000,NaN,5.1,fa,NaN,NaN,NaN,NaN,...,2018-06-04T20:43:44.000Z,Missouri,earthquake,NaN,NaN,NaN,NaN,reviewed,ushis,sc
